# Tarea

Implementar el método de esquemas del trapecio, para valuar la opción call y put asiática con precio inicial, $S_0 = 100$, precio de ejercicio $K = 100$, tasa libre de riesgo $r = 0.10$, volatilidad $\sigma = 0.20$ y $T = 1$ año. Cuyo precio es $\approx 7.04$. Realizar la simulación en base a la siguiente tabla:
![imagen.png](attachment:imagen.png)

Observe que en esta tabla se encuentran los intervalos de confianza de la aproximación obtenida y además el tiempo de simulación que tarda en encontrar la respuesta cada método. 
- Se debe entonces realizar una simulación para la misma cantidad de trayectorias y número de pasos y construir una Dataframe de pandas para reportar todos los resultados obtenidos.**(70 puntos)**
- Compare los resultados obtenidos con los resultados arrojados por la función `Riemann_approach`. Concluya. **(30 puntos)**

In [1]:
#importar los paquetes que se van a usar
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
%matplotlib inline
#algunas opciones para Pandas
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 9)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 78)
pd.set_option('precision', 3)

In [2]:
def BSprices(mu,sigma,S0,NbTraj,NbStep):
    T = 1
    nu = mu-(sigma**2)/2
    
    DeltaT = T/NbStep
    SqDeltaT = np.sqrt(DeltaT)
    
    #for i in range(NbStep):
    DeltaW = SqDeltaT*np.random.randn(NbTraj,NbStep-1)
    increments = nu*DeltaT + sigma*DeltaW
    concat = np.concatenate((np.log(S0)*np.ones([NbTraj,1]),increments),axis=1)
    LogSt = np.cumsum(concat,axis=1)
    St = np.exp(LogSt)
    t = np.arange(0,1,DeltaT)

    return St.T,t

In [3]:
def calc_daily_ret(closes):
    return np.log(closes/closes.shift(1)).iloc[1:]

In [4]:
def temp(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad'):
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    t = t*NbStep
    
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    
    # Obtenemos los precios promedios
    #Average_t = prices.expanding().mean()
    
    vnormal = np.random.randn(NbStep,NbTraj)*sigma+2+r*(T/NbStep)
    suma = np.cumsum(np.multiply(prices,vnormal)*((T/NbStep)/(2*T)))
    
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    return suma,strike,t

# CALL

In [5]:
def TrapC(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad'):
 
    # Resolvemos la ecuación de black scholes para obtener los precios
    
    suma,strike,t=temp(K,r,S0,NbTraj,NbStep,sigma)
    # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann #cambiaaaa
    call = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(suma-strike,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
    
    # intervalos de confianza
    confianza = 0.95
    sigma_est = call.sem().Prima
    mean_est = call.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
    
    #return np.array([call.iloc[-1].Prima,i1[0],i1[1]])
    return call.iloc[-1].Prima

In [6]:
NbTraj = [1000,5000,10000,50000,100000,500000,1000000]
NbStep = [10,50,100]

S0 = 100     # Precio inicial
r = 0.10     # Tasa libre de riesgo 
sigma = 0.2  # volatilidad
K = 100      # Strike price
T = 1        # Tiempo de cierre - años

# Call = np.zeros([len(NbTraj),len(NbStep)])
# intervalos = []#np.zeros([len(NbTraj),len(NbStep)])
M = list(map(lambda N_tra:list(map(lambda N_ste:TrapC(K,r,S0,N_tra,N_ste,sigma),NbStep)),NbTraj))
M = np.asmatrix(M)

MemoryError: 

In [ ]:
# Visualización de datos 
filas = ['Nbtray = %i' %i for i in NbTraj]
col = ['NbStep = %i' %i for i in NbStep]
df = pd.DataFrame(index=filas,columns=col)
df.loc[:,:] = M
df

# PUT

In [ ]:
def TrapP(K:'Strike price',r:'Tasa libre de riesgo',S0:'Precio inicial',
                     NbTraj:'Número trayectorias',NbStep:'Cantidad de pasos a simular',
                     sigma:'Volatilidad'):
    
    # Resolvemos la ecuación de black scholes para obtener los precios
    St,t = BSprices(r,sigma,S0,NbTraj,NbStep)
    t = t*NbStep
    
    # Almacenamos los precios en un dataframe
    prices = pd.DataFrame(St,index=t)
    
    # Obtenemos los precios promedios
    #Average_t = prices.expanding().mean()
    vnormal = np.random.randn(NbStep,NbTraj)*sigma+2+r*(T/NbStep)
    suma = np.cumsum(np.multiply(prices,vnormal)*((T/NbStep)/(2*T)))
    
    # Definimos el dataframe de strikes
    strike = pd.DataFrame(K*np.ones([NbStep,NbTraj]), index=t)
    
    # Calculamos el call de la opción según la formula obtenida para Sumas de Riemann #cambiaaaa
    put = pd.DataFrame({'Prima':np.exp(-r*T) \
                 *np.fmax(strike-suma,np.zeros([NbStep,NbTraj])).mean(axis=1)}, index=t)
    
    # intervalos de confianza
    confianza = 0.95
    sigma_est = put.sem().Prima
    mean_est = put.iloc[-1].Prima
    i1 = st.norm.interval(confianza, loc=mean_est, scale=sigma_est)
    
#     return np.array([call.iloc[-1].Prima,i1[0],i1[1]])
    return put.iloc[-1].Prima

In [ ]:
M = list(map(lambda N_tra:list(map(lambda N_ste:TrapP(K,r,S0,N_tra,N_ste,sigma),NbStep)),NbTraj))
M = np.asmatrix(M)

In [ ]:
# Visualización de datos 
filas = ['Nbtray = %i' %i for i in NbTraj]
col = ['NbStep = %i' %i for i in NbStep]
df = pd.DataFrame(index=filas,columns=col)
df.loc[:,:] = M
df

In [ ]:
M